In [1]:
rm(list=ls())

source("./ins_paquetes.R")
ins_paquetes('MLmetrics','reshape','data.table','dplyr',
             'party','lattice','sqldf','ggplot2','mlr',
             'corrplot','haven','pROC','reshape2')

Warning message:
"package 'MLmetrics' was built under R version 3.6.3"
Attaching package: 'MLmetrics'

The following object is masked from 'package:base':

    Recall

Warning message:
"package 'reshape' was built under R version 3.6.3"Warning message:
"package 'data.table' was built under R version 3.6.3"
Attaching package: 'data.table'

The following object is masked from 'package:reshape':

    melt


Attaching package: 'dplyr'

The following objects are masked from 'package:data.table':

    between, first, last

The following object is masked from 'package:reshape':

    rename

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'party' was built under R version 3.6.3"Warning message:
"package 'mvtnorm' was built under R version 3.6.3"Warning message:
"package 'modeltools' was built under R version 3.6.3"Warning message:
"package 'struccha


Bellow Packages Successfully Installed:

 MLmetrics    reshape data.table      dplyr      party    lattice      sqldf 
      TRUE       TRUE       TRUE       TRUE       TRUE       TRUE       TRUE 
   ggplot2        mlr   corrplot      haven       pROC   reshape2 
      TRUE       TRUE       TRUE       TRUE       TRUE       TRUE 


In [2]:
getwd()
setwd('C:/Users/Aron/Desktop/Analitica/S5')

[1] "C:/Users/Aron/Desktop/Analitica/S5"

In [3]:
library(dplyr)
library(MLmetrics)
library(corrplot)
library(mlr)
library(sqldf)
library(ggplot2)
library(haven)
library(pROC)
library(reshape2)

In [4]:
datos <- haven::read_sav("./datasets/Churn.sav")
datos <- data.frame(datos)
head(datos)

ID,EDAD,SEXO,CIVIL,HIJOS,INGRESO,AUTO,CHURN
0,57,1,1,2,27535.3,1,1
4,68,1,1,2,81000.9,2,1
8,34,2,2,0,87467.1,1,0
10,60,2,1,2,83220.6,2,1
13,28,1,1,2,20850.4,2,1
17,52,2,2,0,84112.6,2,0


In [5]:
########### 3) TRATAMIENTO DE LA DATA ################# 

## en primer lugar ver el analisis descriptivo de la data

datos$SEXO <- as.factor(datos$SEXO)
datos$CIVIL <- as.factor(datos$CIVIL)
datos$AUTO <- as.factor(datos$AUTO)
datos$CHURN <- as.factor(datos$CHURN)

resumen <- data.frame(mlr::summarizeColumns(datos))
write.csv(resumen,"output/tabla_resumen_logistica.csv")
resumen

name,type,na,mean,disp,median,mad,min,max,nlevs
ID,numeric,0,2.538752e+03,1.459052e+03,2595.0,1851.7674,0.000,4999.0,0
EDAD,numeric,0,5.745279e+01,2.270700e+01,58.0,28.1694,18.000,97.0,0
SEXO,factor,0,NA,4.996283e-01,NA,NA,672.000,673.0,2
CIVIL,factor,0,NA,4.052045e-01,NA,NA,545.000,800.0,2
HIJOS,numeric,0,9.918216e-01,8.232621e-01,1.0,1.4826,0.000,2.0,0
INGRESO,numeric,0,5.064071e+04,2.835441e+04,50290.7,36315.1009,110.284,99832.9,0
AUTO,factor,0,NA,3.152416e-01,NA,NA,424.000,921.0,2
CHURN,factor,0,NA,3.814126e-01,NA,NA,513.000,832.0,2


In [6]:
########### 4)  ANALISIS  ###################### 

## ver correlacion antes de categorizar

source("./funciones.R")

## Generamos una copia para realizar la correlacion
data_datos <- datos
data_datos[,1:ncol(data_datos)] <- lapply(data_datos[,1:ncol(data_datos)],as.character)
data_datos[,1:ncol(data_datos)] <- lapply(data_datos[,1:ncol(data_datos)],as.numeric)

corre <- cor(data_datos,method = c("spearman"))

## colocamos la primera funcion de correlacion
corre <- correlacionS(corre)
corre$filtro <- ifelse(abs(corre$cor)>0.6,1,0)

write.csv(corre,"output/correlacion_variables_logistica.csv",row.names = F)

In [7]:
########### 4)  ANALISIS  ###################### 

## ver correlacion antes de categorizar

source("./funciones.R")

## Generamos una copia para realizar la correlacion
data_datos <- datos
data_datos[,1:ncol(data_datos)] <- lapply(data_datos[,1:ncol(data_datos)],as.character)
data_datos[,1:ncol(data_datos)] <- lapply(data_datos[,1:ncol(data_datos)],as.numeric)

corre <- cor(data_datos,method = c("spearman"))

## colocamos la primera funcion de correlacion
corre <- correlacionS(corre)
corre$filtro <- ifelse(abs(corre$cor)>0.6,1,0)

write.csv(corre,"output/correlacion_variables_logistica.csv",row.names = F)

In [8]:
## primer modelo logistica ##

# retiramos la variable correlacionada

data.frame(names(datos))

formula <-    CHURN ~
         EDAD+
         SEXO+
        CIVIL+
        HIJOS+
      INGRESO+
         AUTO
        
              
modelo1 <- glm(formula,data=datos,family = binomial() )

names.datos.
ID
EDAD
SEXO
CIVIL
HIJOS
INGRESO
AUTO
CHURN


In [9]:
# indicadores mediante la segunda funcion
calcula_indicadores(modelo1)

Setting levels: control = 0, case = 1
Setting direction: controls < cases


variable,Gini,KS
logistico,58.94,56.04


In [10]:
# ver la significancia del modelo
summary(modelo1)


Call:
glm(formula = formula, family = binomial(), data = datos)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8069  -0.6306  -0.4777   0.9101   2.3761  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  5.342e-01  2.553e-01   2.092 0.036410 *  
EDAD         1.048e-02  2.943e-03   3.561 0.000370 ***
SEXO2       -2.485e+00  1.428e-01 -17.406  < 2e-16 ***
CIVIL2       3.088e-02  1.349e-01   0.229 0.818955    
HIJOS       -1.058e-01  8.079e-02  -1.310 0.190139    
INGRESO     -8.119e-06  2.356e-06  -3.446 0.000568 ***
AUTO2       -1.072e-01  1.423e-01  -0.753 0.451262    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1788.2  on 1344  degrees of freedom
Residual deviance: 1391.3  on 1338  degrees of freedom
AIC: 1405.3

Number of Fisher Scoring iterations: 4


In [11]:
## reajustando
formula <-    CHURN ~
  EDAD+
  SEXO+
  #CIVIL+
  #HIJOS+
  INGRESO
  #AUTO

modelo2 <- glm(formula,data=datos,family = binomial())

In [12]:
# indicadores mediante la segunda funcion
calcula_indicadores(modelo2)

Warning message in ks.test(x = objeto_logit$fitted.values[which(objeto_logit$y == :
"p-value will be approximate in the presence of ties"Setting levels: control = 0, case = 1
Setting direction: controls < cases


variable,Gini,KS
logistico,58.62,56.43


In [13]:
# ver la significancia del modelo
summary(modelo2)


Call:
glm(formula = formula, family = binomial(), data = datos)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7474  -0.6227  -0.4799   0.9218   2.3539  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  3.528e-01  2.131e-01   1.655 0.097863 .  
EDAD         1.057e-02  2.935e-03   3.601 0.000316 ***
SEXO2       -2.475e+00  1.423e-01 -17.391  < 2e-16 ***
INGRESO     -7.994e-06  2.352e-06  -3.398 0.000678 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1788.2  on 1344  degrees of freedom
Residual deviance: 1393.7  on 1341  degrees of freedom
AIC: 1401.7

Number of Fisher Scoring iterations: 4


In [14]:
## reajustando

formula <-    CHURN ~
  EDAD+
  SEXO+
  #CIVIL+
  #HIJOS+
  INGRESO-1
#AUTO

modelo3 <- glm(formula,data=datos,family = binomial())

In [15]:
# indicadores mediante la segunda funcion
calcula_indicadores(modelo3)

Warning message in ks.test(x = objeto_logit$fitted.values[which(objeto_logit$y == :
"p-value will be approximate in the presence of ties"Setting levels: control = 0, case = 1
Setting direction: controls < cases


variable,Gini,KS
logistico,58.62,56.43


In [16]:
# ver la significancia del modelo
summary(modelo3)


Call:
glm(formula = formula, family = binomial(), data = datos)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7474  -0.6227  -0.4799   0.9218   2.3539  

Coefficients:
          Estimate Std. Error z value Pr(>|z|)    
EDAD     1.057e-02  2.935e-03   3.601 0.000316 ***
SEXO1    3.528e-01  2.131e-01   1.655 0.097863 .  
SEXO2   -2.123e+00  2.348e-01  -9.040  < 2e-16 ***
INGRESO -7.994e-06  2.352e-06  -3.398 0.000678 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1864.6  on 1345  degrees of freedom
Residual deviance: 1393.7  on 1341  degrees of freedom
AIC: 1401.7

Number of Fisher Scoring iterations: 4


In [ ]:
## representacion de las variables

datosF <- sqldf::sqldf("select   EDAD,  SEXO,  INGRESO from datos")
datosF$probabilidades_result1 <- predict(modelo3, newdata = datos, type = "response")

tabla <- reshape2::melt(datosF, id.vars = c("EDAD", "SEXO","INGRESO"), value.name = "probability")

# Grafico de las probabilidades

ggplot2::ggplot(tabla, aes(x = EDAD, 
                           y = probability, colour = SEXO)) + geom_line() + 
  facet_grid(variable ~., scales = "free")

ggplot2::ggplot(tabla, aes(x = INGRESO, 
                           y = probability, colour = SEXO)) + geom_line() + 
  facet_grid(variable ~., scales = "free")

## Indicadores

## calcular el AUC
AUC1 <- MLmetrics::AUC(datosF$probabilidades_result1,datos$CHURN)

## calcular el GINI
gini1 <- 2*(AUC1) -1

## calcular el LOGLOSS
LogLoss1 <- MLmetrics::LogLoss(datosF$probabilidades_result1,as.numeric(as.character(datos$CHURN)))

## Calcular el KS
KS1 <- MLmetrics::KS_Stat(datosF$probabilidades_result1,datos$CHURN)

# Calcular los valores predichos

table(datos$CHURN)

PRED <- ifelse(datosF$probabilidades_result1<=0.5,0,1) # pto de corte 0.5
#PRED <- ifelse(datosF$probabilidades_result1<=mean(datosF$probabilidades_result1),0,1)
PRED <- as.factor(PRED)

# Calcular la matriz de confusion
tabla <- caret::confusionMatrix(PRED,datos$CHURN,positive = "1")
tabla

# sensibilidad
Sensitivity1 <- MLmetrics::Sensitivity(PRED,datos$CHURN)

#Specificity
Specificity1 <- MLmetrics::Specificity(PRED,datos$CHURN)

# Precision
Accuracy1 <- MLmetrics::Accuracy(PRED,datos$CHURN)

# Calcular el error de mala clasificaci?n
error1 <- 1-Accuracy1

# indicadores
AUC1
gini1
LogLoss1
KS1
Accuracy1
error1
Sensitivity1
Specificity1

########### 5) SEGUNDO ANALISIS  #################

formula <-    CHURN ~
  EDAD+
  SEXO+
  CIVIL+
  HIJOS+
  INGRESO+
  AUTO


modelo4 <- glm(formula,data=datos,family=binomial(link=cloglog) )

# indicadores mediante la segunda funcion
calcula_indicadores(modelo4)

# ver la significancia del modelo

summary(modelo4)

## reajustando

formula <-    CHURN ~
  EDAD+
  SEXO+
  #CIVIL+
  #HIJOS+
  INGRESO#+
  #AUTO


modelo5 <- glm(formula,data=datos,family=binomial(link=cloglog) )

# indicadores mediante la segunda funcion
calcula_indicadores(modelo5)

# ver la significancia del modelo

summary(modelo5)

## reajustando

formula <-    CHURN ~
  EDAD+
  SEXO+
  #CIVIL+
  #HIJOS+
  INGRESO -1 #+
#AUTO


modelo6 <- glm(formula,data=datos,family=binomial(link=cloglog) )

# indicadores mediante la segunda funcion
calcula_indicadores(modelo6)

# ver la significancia del modelo

summary(modelo6)

## Indicadores

datosF$probabilidades_result2 <- predict(modelo6, newdata =  datos,type="response")

## calcular el AUC
AUC2 <- MLmetrics::AUC(datosF$probabilidades_result2,datos$CHURN)

## calcular el GINI
gini2 <- 2*(AUC2) -1

## calcular el LOGLOSS
LogLoss2 <- MLmetrics::LogLoss(datosF$probabilidades_result2,as.numeric(as.character(datos$CHURN)))

## Calcular el KS
KS2 <- MLmetrics::KS_Stat(datosF$probabilidades_result2,datos$CHURN)

# Calcular los valores predichos

PRED <- ifelse(datosF$probabilidades_result2<=0.5,0,1) # pto de corte 0.5
PRED <- as.factor(PRED)

# Calcular la matriz de confusion
tabla <- caret::confusionMatrix(PRED,datos$CHURN,positive = "1")
tabla

# sensibilidad
Sensitivity2 <- MLmetrics::Sensitivity(PRED,datos$CHURN)

#Specificity
Specificity2 <- MLmetrics::Specificity(PRED,datos$CHURN)

# Precision
Accuracy2 <- MLmetrics::Accuracy(PRED,datos$CHURN)

# Calcular el error de mala clasificaci?n
error2 <- 1-Accuracy2

# indicadores
AUC2
gini2
LogLoss2
KS2
Accuracy2
error2
Sensitivity2
Specificity2

## Tabla comparativa

AUC <- rbind(AUC1,AUC2); AUC <- round(AUC,3)
Gini <- rbind(gini1,gini2); Gini <- round(Gini,3)
LogLoss <- rbind(LogLoss1,LogLoss2); LogLoss <- round(LogLoss,4)
KS <- rbind(KS1,KS2); KS <- round(KS,4)
Accuracy <- rbind(Accuracy1,Accuracy2); Accuracy <- round(Accuracy,3)
Sensitivity <- rbind(Sensitivity1,Sensitivity2); Sensitivity <- round(Sensitivity,3)
Specificity <- rbind(Specificity1,Specificity2); Specificity <- round(Specificity,3)

Indicadores <- cbind(AUC,Gini,LogLoss,KS,Accuracy,Sensitivity,Specificity)
rownames(Indicadores)<- c('modelo_logit','modelo_cloglog')
colnames(Indicadores)<- c('AUC','Gini','LogLoss','KS','Accuracy','Sensitivity','Specificity')

Indicadores
